# Name standardization

One of the major problems in any data-handling situation is the need to match up databases based on names as indices may either not be available or different databases on different systems use incompatible identifiers. An example for insurance is the matching of company names and the logic for storing these names is not consistent (e.g. company names may change or be misspelt and the extension may be abbreviated and the abbreviations are used, such as "Limited" and "Ltd" or "GmbH" and "Gesellschaft mbH" and "G.m.b.H"). 

In order to perform matching we will have to rely soft or fuzzy matching but we will need to do some cleaning to help the algorithm further, such as tokenizing strings and removing special characters and punctuations.

This notebook provides demonstration only and we import a set of artificial data that contains messy information, i.e. each employee ID is associated to a company but we state in actuality the subsidiaries. These names are messy and we want to tidy it up into the two companies:

- XYZ Specialty
- ABC Solutions

See the Section "Apply to external" to see how this performs on an CSV file.

A lot of the ideas come from the following:
https://www.analyticsinsight.net/company-names-standardization-using-a-fuzzy-nlp-approach/ 

In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process
from sklearn import cluster
from difflib import SequenceMatcher
import re
from collections import Counter

import os


## Text cleaning
We introduce a few functions that deal with removing special characters by turning them all into spaces and then also removing any duplicated spaces. 

In [2]:
def remove_special_chars(string_in: str) -> str:
    '''
    Remove various bits of punctuation from input and returns all in lower case

    Input:
        string_in - str
    
    Output:
        string_out - str
    '''
    chars = [' ', ',', ';', ':', '.', ',', '*', '#', '@', '|', '/', '\\', '-', '_', '?', '%', '!', '^', '(', ')']
    default_char = chars[0]

    tmp_str = string_in
    for c in chars[1:]:
        tmp_str = tmp_str.replace(c, default_char)

    # We also remove multiple spaces by using re.sub
    string_out = re.sub(' +', ' ', tmp_str)
    return string_out.lower()

# Test out this function
str_test = 'This-:**string@has##no(bad!_______characters%?whatsoever'
test_out = remove_special_chars(str_test)
print(test_out)

this string has no bad characters whatsoever


## Matching and simplifying
The FuzzyWuzzy library can be used to to evaluate text similarity by considering Levenshtein distances (see: https://en.wikipedia.org/wiki/Levenshtein_distance). We also want to allow for if the name is presented in different orders. This cam be done by considering the harmonic mean of *partial_ratio* and *token_set_ratio* metrics from FuzzyWuzzy, call this the Similarity Score. The harmonic mean as we are considering average ratios. 

This can be visualized in a matrix, $S$, where each element $S_{i,j}$ is the Similarity Score betwween the $i$-th and $j$-th names.

Next, we perform clustering on this matrix, which creates clusters of names potentially belonging to the same company. We choose *Affinity Propagation* as even the number of clusters can be chosen based on data, as opposed to $K$ means where the hyperparameter of the number of clusters is needed.

When the clusters are assigned, we then consider all pairs of names and by using the longst common substring, we define this as our 'Company name'.

Finally, to evaluate the success of our algorithm, we want to measure confidence of our matchings, whereby we compare the cleaned string against the standard string. 

In [3]:
def generate_similarity_score_matrix(stringVect):
    '''
    Generate the matrix of Similarity Score for each pair of strings given. The Similarity Score is the harmonic mean of FuzzyWuzzy's token_set_ratio and partial_ratio parameters

    Requires:
        fuzzywuzzy.fuzz
    
    Input:
        stringVect - a vector of strings
    
    Output:
        S - a (n,n)-matrix of Similarity Scores where n = len(stringVect)
    '''
    N=len(stringVect)
    S=np.ones((N,N))*100

    # Generater the entries - we add a tiny number to avoid possible divisions by 0 during ou harmonic mean calcs
    for i in range(N):
        for j in range(N):
            s1 = fuzz.token_set_ratio(stringVect[i], stringVect[j]) + 1e-10
            s2 = fuzz.partial_ratio(stringVect[i], stringVect[j]) + 1e-10
            S[i,j] = 2*s1*s2/(s1+s2)
    return S

# Test the above function - main diagonal should be 100
vectTest = ['abc', 'abd', 'cba', 'xyz']
testOut = generate_similarity_score_matrix(vectTest)
similarity_df = pd.DataFrame(data=testOut, columns=vectTest, index=vectTest).round(0)
similarity_df

,abc,abd,cba,xyz
abc,100.0,67.0,33.0,0.0
abd,67.0,100.0,33.0,0.0
cba,33.0,33.0,100.0,0.0
xyz,0.0,0.0,0.0,100.0


In [4]:
def names_clustering(stringVect):
    '''
    Create clusters of most commonly appearing sub-strings and assign them to items passed in.

    Clustering is done on the similarity matrix, which we will call here on our input

    Requires:
        sklearn.AffinityPropagation
        fuzzywuzzy.fuzz

    Input:
        stringVect - vector of strings
    
    Output:
        dfCluster - a dataframe that contains the original stringVect inputs and their associated cluster
    '''

    # Generate the similarity matrix on input
    S = generate_similarity_score_matrix(stringVect)

    # Fit the Affinity Propagation clustering algorithm on similarity matrix, S
    clusters = cluster.AffinityPropagation(affinity='precomputed', random_state=None).fit_predict(S)

    # Create the output dataframe
    dfCluster = pd.DataFrame(list(zip(stringVect, clusters)), columns=['input_names', 'cluster'])
    return dfCluster

testInput = ['abcd enterprise', 'abcd solutions', 'abcd Europe', 'abcd Asia', 'xyzp America', 'xyzp Portugal', 'xyzp Holdings']
out = names_clustering(testInput)
out

,input_names,cluster
0,abcd enterprise,0
1,abcd solutions,0
2,abcd Europe,0
3,abcd Asia,0
4,xyzp America,1
5,xyzp Portugal,1
6,xyzp Holdings,1


As we can see in the above testing, the algorithm has detected two clusters, which is correct given we wanted the algorithm to detect 'abcd' and 'xyzp'. These are enumerated into 0 and 1 respectively. 

We now need to find a suitable label for each of these clusters

In [5]:
def get_standard_name(dfClustered, namesCol='input_names'):
    '''
    Names each generated cluster according to the longest common substring in each cluster. Multiple modes will be added if present.
    
    Requires:
        difflib.SequenceMatcher
        collections.Counter
        fuzzywuzzy.fuzz

    Input:
        dfCluster - needs column name "cluster"!
        namesCol - the column with which to search for names 
        
    Output:
        dfNamedCluster - similar to dfCluster but with a suitable name.
    '''
    # Clean up the input_names
    dfClustered[namesCol] = dfClustered[namesCol].apply(remove_special_chars)

    # Initialize empty dictionary - we will fill with the cluster enumeration
    dict_standard_names = {}

    for _cluster in dfClustered['cluster'].unique():
        # Filter in in this cluster, retrive the names column (namesCol)
        names = dfClustered[dfClustered['cluster']==_cluster][namesCol].to_list()

        lsCommonSubstring = [] # Pre-init list that will store common substring

        if len(names) == 1:
            # In this trivial case, we just use the actual name here as our 'common substring'
            dict_standard_names[_cluster] = names[0]
        elif len(names) == 0:
            # This should never need executing but in case there is blanks, then we return string 'unknown'
            dict_standard_names[_cluster] = 'Unknown'
        else:
            # In this cluster, compare pairwise matches and obtain the longest in each pair
            for i in range(len(names)):
                for j in range(i+1, len(names)):
                    sequenceMatch = SequenceMatcher(None, names[i], names[j])
                    match = sequenceMatch.find_longest_match(0, len(names[i]), 0, len(names[j]))

                    # Add the common bits as a candidate to the list 'lsCommonSubstring' if non-empty
                    if (match.size > 0):
                        lsCommonSubstring.append(names[i][match.a:match.a + match.size].strip())
            
            n_common = len(lsCommonSubstring)
            counts = Counter(lsCommonSubstring)
            get_mode = dict(counts)
            mode = [k for k, v in get_mode.items() if v == max(list(counts.values()))]
            dict_standard_names[_cluster] = ';'.join(mode)
    
    # Tidy up the dictionary of standard names into a dataframe
    df_standard_names = pd.DataFrame(list(dict_standard_names.items()), columns=['cluster', 'standard_name'])

    # Join with the input data
    df = pd.merge(dfClustered, df_standard_names, on='cluster', how='left')

    # Also add in the confidence level
    df['Confidence'] = df.apply(lambda x: fuzz.token_set_ratio(x['standard_name'], x[namesCol]), axis=1)

    return df
    

# To test, we use the output from the previous cell
result = get_standard_name(out, namesCol='input_names')
result

,input_names,cluster,standard_name,Confidence
0,abcd enterprise,0,abcd,100
1,abcd solutions,0,abcd,100
2,abcd europe,0,abcd,100
3,abcd asia,0,abcd,100
4,xyzp america,1,xyzp,100
5,xyzp portugal,1,xyzp,100
6,xyzp holdings,1,xyzp,100


As we can see, now we have the clustering completed correctly and a standard name introduced that map to 'abcd' and 'xyzp' respectively. Given these standard names are substrings of input_names, we have 100 confidence.

## Apply to external
We have an external CSV file ready to use. It contains employee IDs and messy company names. We want to clean this up and then assign the employee ID to the correct cleaned up company.

To do so, we use the following workflow

1. names_clustering(text to tidy)
2. get_standard_name(result from step 1, the name of the column to use)
3. Merge the new standard names back to the original data set

In [6]:
# Ensure the working directory is called 'Name_standardizations' - may not be the case if we use the git top level directory
if not os.getcwd().endswith('Name_standardizations'):
    os.chdir('Notebooks/Name_standardizations')
data = pd.read_csv('messy_data.csv')

data.head(10)

,Employee ID,Company
0,44238,xyz specialty Ltd
1,98684,xyz specialty?Holdings
2,60741,xyz specialty company Southeast
3,39180,xyz specialty nw
4,32634,xyz specialty GmbH
5,37998,specialtyz Ltd xyz
6,11211,xyz specialty Services Oceania
7,52416,abc solutions topco
8,62553,xyz specialty Subsidiary
9,4140,xyz specialty pd


In [7]:
d1 = names_clustering(data['Company'])
d1

,input_names,cluster
0,xyz specialty Ltd,0
1,xyz specialty?Holdings,0
2,xyz specialty company Southeast,0
3,xyz specialty nw,0
4,xyz specialty GmbH,0
5,specialtyz Ltd xyz,0
6,xyz specialty Services Oceania,0
7,abc solutions topco,1
8,xyz specialty Subsidiary,0
9,xyz specialty pd,0


In [8]:
d2 = get_standard_name(d1)
d2

,input_names,cluster,standard_name,Confidence
0,xyz specialty ltd,0,xyz specialty,100
1,xyz specialty holdings,0,xyz specialty,100
2,xyz specialty company southeast,0,xyz specialty,100
3,xyz specialty nw,0,xyz specialty,100
4,xyz specialty gmbh,0,xyz specialty,100
5,specialtyz ltd xyz,0,xyz specialty,84
6,xyz specialty services oceania,0,xyz specialty,100
7,abc solutions topco,1,solutions,100
8,xyz specialty subsidiary,0,xyz specialty,100
9,xyz specialty pd,0,xyz specialty,100


In [9]:
data['cleaned_names'] = data['Company'].apply(remove_special_chars)
d3 = pd.merge(data, d2[['input_names', 'standard_name', 'Confidence']], left_on='cleaned_names', right_on='input_names', how='left')
d3.drop(['cleaned_names', 'input_names'], axis=1)

,Employee ID,Company,standard_name,Confidence
0,44238,xyz specialty Ltd,xyz specialty,100
1,98684,xyz specialty?Holdings,xyz specialty,100
2,98684,xyz specialty?Holdings,xyz specialty,100
3,60741,xyz specialty company Southeast,xyz specialty,100
4,39180,xyz specialty nw,xyz specialty,100
5,32634,xyz specialty GmbH,xyz specialty,100
6,37998,specialtyz Ltd xyz,xyz specialty,84
7,11211,xyz specialty Services Oceania,xyz specialty,100
8,52416,abc solutions topco,solutions,100
9,62553,xyz specialty Subsidiary,xyz specialty,100


## Final thoughts
One major flaw that this algorithm may overlook is if there are two companies that have very similar names. They may become classed as one company. E.g it could be argued that 'Abc-PKP Solutions' could be another company that is not part of 'ABC Solutions'. Here the model sees the common name 'solutions' and so takes that as the top-level name.